In [1]:
NODE_COUNT = 8
LOG_DIR = f"/home/<User>/repos/csg_is/calculating_scores/DFL/non_iid/fedstellar_DFL_21_07_2024_14_58_30/metrics/participant_"
date_str = "21/07/2024 12:58:30	" # minus two hours for timezone

In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(NODE_COUNT):
    # Directory containing your event files
    log_dir = LOG_DIR +f"{i}"
    print(log_dir)

    date_obj = datetime.strptime(date_str.rstrip(" ").rstrip("	"), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

/home/<User>/repos/csg_is/calculating_scores/DFL/non_iid/fedstellar_DFL_21_07_2024_14_58_30/metrics/participant_0
   step      value                     timestamp                    tag
0  6898  33.799999 2024-07-21 13:18:34.137126656  Resources/CPU_percent
1  6908   4.400000 2024-07-21 13:18:44.601262848  Resources/CPU_percent
2  8132  93.199997 2024-07-21 13:18:54.631870720  Resources/CPU_percent
3  8142  99.699997 2024-07-21 13:19:04.683599616  Resources/CPU_percent
4  8152  99.599998 2024-07-21 13:19:14.718110464  Resources/CPU_percent
2024-07-21 13:19:24 2024-07-21 12:58:43
                step     value                     timestamp
tag                                                         
Test/Accuracy   6908  0.839710 2024-07-21 13:19:22.074529536
Test/F1Score    6908  0.793297 2024-07-21 13:19:22.074564096
Test/Loss       6908  0.933723 2024-07-21 13:19:22.074497536
Test/Precision  6908  0.782154 2024-07-21 13:19:22.074542336
Test/Recall     6908  0.839710 2024-07-21 13:19:

In [3]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,6908,0.839710,2024-07-21 13:19:22.074529536,participant_0
Test/F1Score,6908,0.793297,2024-07-21 13:19:22.074564096,participant_0
Test/Loss,6908,0.933723,2024-07-21 13:19:22.074497536,participant_0
Test/Precision,6908,0.782154,2024-07-21 13:19:22.074542336,participant_0
Test/Recall,6908,0.839710,2024-07-21 13:19:22.074553344,participant_0
Test/Accuracy,6909,0.839710,2024-07-21 13:19:21.935661056,participant_1
Test/F1Score,6909,0.793297,2024-07-21 13:19:21.935692800,participant_1
Test/Loss,6909,0.933723,2024-07-21 13:19:21.935628032,participant_1
Test/Precision,6909,0.782154,2024-07-21 13:19:21.935672832,participant_1


In [4]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_23298/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,7768.25,0.8397
Test/F1Score,7768.25,0.7933
Test/Loss,7768.25,0.9337
Test/Precision,7768.25,0.7822
Test/Recall,7768.25,0.8397


In [5]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_23298/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,1603.16,0.0
Test/F1Score,1603.16,0.0
Test/Loss,1603.16,0.0
Test/Precision,1603.16,0.0
Test/Recall,1603.16,0.0


In [6]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

/tmp/ipykernel_23298/1227319542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_results = concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,7768.25,0.8397
Test/F1Score,7768.25,0.7933
Test/Loss,7768.25,0.9337
Test/Precision,7768.25,0.7822
Test/Recall,7768.25,0.8397
